In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from TX import bcc_rolling_fiber
import time

- Create fiber textures in various intensity levels
by applying different weights of $\gamma$-fiber ($\chi^\chi$) and random textures ($\chi^\text{random}$).

$\chi^\text{total}$=$f \chi^\gamma$+$(1-f)\chi^\text{random}$

In [3]:
%%time
polyXtals=[]
fraction = np.linspace(0,1,100)
for i in xrange(len(fraction)):
    gammaXtal=bcc_rolling_fiber.main(
        ngrains=500,
        fiber='gamma',
        sigma=15,iexit=True)
    frac1 = fraction[i]
    frac2 = 1.-frac1
    randomXtal=bcc_rolling_fiber.main(
        ngrains=500,
        fiber='random',
        iexit=True)
    gammaXtal[:,-1]=gammaXtal[:,-1]*frac1
    randomXtal[:,-1]=randomXtal[:,-1]*frac2
    pxtal = np.concatenate((gammaXtal,randomXtal),axis=0)
    polyXtals.append(pxtal)

** joblib was not found - will not be used in TX.upf
CPU times: user 9.94 s, sys: 44.8 ms, total: 9.99 s
Wall time: 9.99 s


- Plot pole figures

In [4]:
from TX import upf ## load pole figure plotter

- Plot (111) pole figure and save them to /tmp/pf_???.png files

In [5]:
%%time
for i in xrange(100):
    mypf = upf.polefigure(grains=polyXtals[i],csym='cubic')
    fig=mypf.pf_new(poles=[[1,1,1]],cmap='jet',nlev=7,mn=0.8,mx=10)
    fig.savefig('/tmp/pf_%3.3i.png'%i,bbox_inches='tight',dpi=300)
    fig.clf()
    plt.close(fig)

Elapsed time for calling cells_pf:          93 [ ms]
Elapsed time for calling cells_pf:          88 [ ms]
Elapsed time for calling cells_pf:         102 [ ms]
Elapsed time for calling cells_pf:          92 [ ms]
Elapsed time for calling cells_pf:          89 [ ms]
Elapsed time for calling cells_pf:          87 [ ms]
Elapsed time for calling cells_pf:          89 [ ms]
Elapsed time for calling cells_pf:          94 [ ms]
Elapsed time for calling cells_pf:          86 [ ms]
Elapsed time for calling cells_pf:          92 [ ms]
Elapsed time for calling cells_pf:          98 [ ms]
Elapsed time for calling cells_pf:          88 [ ms]
Elapsed time for calling cells_pf:          88 [ ms]
Elapsed time for calling cells_pf:          91 [ ms]
Elapsed time for calling cells_pf:          92 [ ms]
Elapsed time for calling cells_pf:          88 [ ms]
Elapsed time for calling cells_pf:          88 [ ms]
Elapsed time for calling cells_pf:          87 [ ms]
Elapsed time for calling cells_pf:          89

/usr/local/lib/python2.7/site-packages/matplotlib/contour.py:1518: UserWarning: Log scale: values of z <= 0 have been masked
  warnings.warn('Log scale: values of z <= 0 have been masked')


CPU times: user 1min 9s, sys: 1.34 s, total: 1min 10s
Wall time: 1min 9s


- Convert pngs to mp4 using imagemagick

In [8]:
import os
from glob import glob
fns=glob('/tmp/pf_???.png')
cmd ='convert -delay 8 -quality 100'
for i in xrange(len(fns)):
    cmd = '%s %s'%(cmd, fns[i])
cmd = '%s %s'%(cmd, 'GammaFibAni.mpg')
os.system(cmd)

32512

- Display the video

In [17]:
import io
import base64
from IPython.display import HTML

video = io.open('gammaFibAni.mpg', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" controls>
             <source src="data:video/mpg;base64,{0}" type="video/mpg" />
             </video>'''.format(encoded.decode('ascii')))

IOError: [Errno 2] No such file or directory: 'gammaFibAni.mpg'

- Convert png files to a GIF movie file

In [10]:
## If there's an existing file
!rm gammaFibAni.gif

rm: gammaFibAni.gif: No such file or directory


In [11]:
!ffmpeg -i /tmp/pf_%03d.png gammaFibAni.gif

ffmpeg version 3.3.2 Copyright (c) 2000-2017 the FFmpeg developers
  built with Apple LLVM version 8.1.0 (clang-802.0.42)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/3.3.2 --enable-shared --enable-pthreads --enable-gpl --enable-version3 --enable-hardcoded-tables --enable-avresample --cc=clang --host-cflags= --host-ldflags= --enable-libmp3lame --enable-libx264 --enable-libxvid --enable-opencl --disable-lzma --enable-vda
  libavutil      55. 58.100 / 55. 58.100
  libavcodec     57. 89.100 / 57. 89.100
  libavformat    57. 71.100 / 57. 71.100
  libavdevice    57.  6.100 / 57.  6.100
  libavfilter     6. 82.100 /  6. 82.100
  libavresample   3.  5.  0 /  3.  5.  0
  libswscale      4.  6.100 /  4.  6.100
  libswresample   2.  7.100 /  2.  7.100
  libpostproc    54.  5.100 / 54.  5.100
Input #0, image2, from '/tmp/pf_%03d.png':
  Duration: 00:00:04.00, start: 0.000000, bitrate: N/A
    Stream #0:0: Video: png, rgba(pc), 1000x814 [SAR 11811:11811 DAR 500:407], 25 fps, 25 tbr, 25 tbn, 

In [12]:
ls *

Animate_PF.ipynb     UPF_EPF.ipynb        gammaFibAni.gif
Mg_texture.pdf       UPF_test.ipynb       ipf.ipynb
Mg_texture.png       dual-phase.pdf
UPF_Algorithm.ipynb  fss.pdf

Timo:
convertODFtoVPSCfiles.ipynb


In [14]:
!open ./